In [1]:
import pandas as pd

In [24]:
dataframe = pd.read_csv("../datasets/final_df.csv")

### Aproximacion 1
Lo primero que vamos a hacer es agrupar por temporadas y jugador y para cada jugador miraremos si se ha lesionado de gravedad, para despues coger y transformar el dataset en que un jugador se lesiona de gravedad teniendo en cuenta su carga de trabajo en la temporada anterior 

s40798-022-00427-w-1.pdf

The UEFA guidelines for epidemiological studies were
used in this study [24], so an injury was considered as a
“situation that occurred during a scheduled training session
or match that caused absence from the next training
session or match”. Similarly, these guidelines were
followed to differentiate the severity of injuries, classifying
them into minor (4–7 days of absence), moderate
(8–28 days of absence) and major (more than 28 days
of absence), but slight injuries were excluded.

Vamos a crear una variable nueva que indique para una lesion si el jugador se lesiono de gravedad, teniendo en cunta que la gravedad son mas de 28 dias de ausencia

In [25]:
def define_gravity(df,name,treshold):
    df[name] = df["days_out"].apply(
        lambda x: int(x.split()[0]) >= treshold
    )
    return df

In [26]:
dataframe = define_gravity(dataframe,"gravity_injury_28",28)

In [27]:
dataframe.head()

,Unnamed: 0,Unnamed: 0_x,player,season_x,injury,from,until,days_out,missed_matches,last_season,...,physic,movement_acceleration,movement_sprint_speed,movement_reactions,movement_balance,power_stamina,power_strength,season,matched_short_name,gravity_injury_28
0,0,0,Manuel Neuer,24/25,Torn muscle fiber,"Mar 6, 2025","Apr 30, 2025",56 days,10,23/24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True
1,1,1,Manuel Neuer,24/25,Rib fracture,"Dec 6, 2024","Dec 31, 2024",26 days,4,23/24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
2,2,2,Manuel Neuer,24/25,Thigh problems,"Sep 18, 2024","Sep 26, 2024",9 days,1,23/24,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,3,3,Manuel Neuer,23/24,infection,"May 27, 2024","May 31, 2024",5 days,-,22/23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,4,4,Manuel Neuer,23/24,Torn muscle fiber,"Mar 20, 2024","Apr 8, 2024",20 days,4,22/23,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


In [28]:
dataframe.drop(columns=["Unnamed: 0","Unnamed: 0_x"],inplace=True)

Este preprocesado consiste en agrupar el dataframe a nivel temporada, es decir para cada jugador y temporada me vas a decir si durante esa temporada sufrio alguna lesion

In [29]:
grouped_df_28 = dataframe.groupby(['player', 'season_x'])['gravity_injury_28'].any().reset_index()

grouped_df_28.rename(columns={
    'season_x': 'season',
    'gravity_injury_28': 'had_major_injury_28'
}, inplace=True)

In [30]:
dataframe_aux = dataframe.drop(columns=["injury","from","until","days_out","missed_matches","gravity_injury_28"]).drop_duplicates()

In [31]:
dataframe_aux.head()

,player,season_x,last_season,season_y,nation,pos,age,born,MP,Starts,...,defending,physic,movement_acceleration,movement_sprint_speed,movement_reactions,movement_balance,power_stamina,power_strength,season,matched_short_name
0,Manuel Neuer,24/25,23/24,23/24,GER,GK,37.0,1986.0,23.0,23.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Manuel Neuer,23/24,22/23,22/23,GER,GK,36.0,1986.0,12.0,12.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Manuel Neuer,22/23,21/22,21/22,GER,GK,35.0,1986.0,28.0,28.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Manuel Neuer,21/22,20/21,20/21,GER,GK,34.0,1986.0,33.0,33.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,Manuel Neuer,18/19,17/18,17/18,GER,GK,31.0,1986.0,3.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
dataframe_study = pd.merge(grouped_df_28,dataframe_aux,how = "left",left_on=['player', 'season'],right_on = ["player","season_x"],suffixes=("_1","_2"))

In [33]:
dataframe_study

,player,season_1,had_major_injury_28,season_x,last_season,season_y,nation,pos,age,born,...,defending,physic,movement_acceleration,movement_sprint_speed,movement_reactions,movement_balance,power_stamina,power_strength,season_2,matched_short_name
0,Aaron Cresswell,18/19,True,18/19,17/18,17/18,ENG,DF,27.0,1989.0,...,73.0,68.0,75.0,76.0,68.0,90.0,82.0,59.0,18/19,Aaron Cresswell
1,Aaron Cresswell,19/20,False,19/20,18/19,18/19,ENG,DF,28.0,1989.0,...,74.0,69.0,74.0,72.0,73.0,85.0,74.0,62.0,19/20,Aaron Cresswell
2,Aaron Cresswell,20/21,False,20/21,19/20,19/20,ENG,DF,29.0,1989.0,...,74.0,67.0,74.0,69.0,73.0,83.0,71.0,61.0,20/21,Aaron Cresswell
3,Aaron Cresswell,24/25,False,24/25,23/24,23/24,ENG,"DF,FW",33.0,1989.0,...,77.0,67.0,66.0,51.0,78.0,75.0,73.0,61.0,24/25,Aaron Cresswell
4,Aaron Mooy,18/19,True,18/19,17/18,17/18,AUS,MF,26.0,1990.0,...,66.0,76.0,67.0,70.0,69.0,73.0,78.0,76.0,18/19,Aaron Mooy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6327,Ștefan Radu,18/19,False,18/19,17/18,17/18,ROU,DF,30.0,1986.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6328,Ștefan Radu,19/20,False,19/20,18/19,18/19,ROU,DF,31.0,1986.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6329,Ștefan Radu,20/21,True,20/21,19/20,19/20,ROU,DF,32.0,1986.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6330,Ștefan Radu,21/22,True,21/22,20/21,20/21,ROU,DF,33.0,1986.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
dataframe_study.to_csv("datasetAgrupado28.csv")

In [35]:
dataframe_study.dtypes

player                  object
season_1                object
had_major_injury_28       bool
season_x                object
last_season             object
                        ...   
movement_balance       float64
power_stamina          float64
power_strength         float64
season_2                object
matched_short_name      object
Length: 62, dtype: object

In [36]:
dataframe_study=pd.read_csv("datasetAgrupado28.csv")

In [42]:
dataframe_study.columns

Index(['Unnamed: 0', 'player', 'season_1', 'had_major_injury_28', 'nation',
       'pos', 'age', 'born', 'MP', 'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A',
       'G-PK', 'PK', 'PKatt', 'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG',
       'PrgC', 'PrgP', 'PrgR', 'Gls90', 'Ast90', 'G+A90', 'G-PK90', 'G+A-PK90',
       'xG90', 'xAG90', 'xG+xAG90', 'npxG90', 'npxG+xAG90', 'Unnamed: 0_y',
       'height_cm', 'weight_kg', 'work_rate', 'body_type', 'pace', 'shooting',
       'passing', 'dribbling', 'defending', 'physic', 'movement_acceleration',
       'movement_sprint_speed', 'movement_reactions', 'movement_balance',
       'power_stamina', 'power_strength'],
      dtype='object')

In [ ]:
dataframe_study.drop(columns=[["Unnamed: 0","season_x","last_season","season_y",'_merge','short_name','long_name',
                              'player_positions','season_2','matched_short_name']],inplace=True)